In [1]:
import os
import time
filename = 'DOT_Traffic_Speeds_NBE.csv'

Pandas

In [2]:
import pandas as pd

In [3]:
start = time.time()
pandasDF = pd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

Time to Completion: 3.0 minutes and 21.698315382003784 seconds.


Dask

In [4]:
import dask.dataframe as dd

In [5]:
start = time.time()
daskDF = dd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

Time to Completion: 0.0 minutes and 0.01800370216369629 seconds.


Modin/Ray

In [6]:
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as mpd

In [7]:
start = time.time()
modinDF = mpd.read_csv(filename)
finish = time.time()
print(f'Time to Completion: {(finish-start)//60} minutes and {((finish-start)%60):2} seconds.')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})



Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2022-12-24 14:08:18,026	INFO worker.py:1545 -- Started a local Ray instance.
(pid=) c:\Users\emily\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\dashboard\agent.py:51: DeprecationWarning: There is no current event loop
(pid=)   aiogrpc.init_grpc_aio()


Time to Completion: 1.0 minutes and 11.34940218925476 seconds.


Data Validation

In [8]:
daskDF = dd.read_csv(filename)

In [9]:
daskDF.head()

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,262,34.80,359,0,06/02/2017 11:41:59 PM,4616319,"40.6332305,-74.016151 40.63391,-74.01613 40.63...",ud_wF|gwbMgCCwATcBr@_BvAqDhGmGtL{AxB}AlAsBt@uB...,BBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616319,Brooklyn,GOW S 9TH STREET - 7TH AVENUE
1,204,55.92,155,0,06/02/2017 11:41:59 PM,4616320,"40.7894406,-73.786291 40.78918,-73.78792 40....",_u}wFhkjaMr@dI~A~HtA|EbEnKxBdHv@~Cv@jGRhCJ~G[p...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616320,Queens,CIP N TNB - Whitestone Expwy S Exit 14 (Linden...
2,106,39.77,159,0,06/02/2017 11:41:59 PM,4616323,"40.77158,-73.994441 40.7713004,-73.99455 40.77...",kezwFf`sbMv@TxAVnDZe@Gz@J~@Xf@VlEnC??~KpH??vCp...,BBBBBBBBBBBBBBBBB,NYC_DOT_LIC,4616323,Manhattan,12th Ave S 57th St - 45th St
3,184,65.24,39,0,06/03/2017 04:46:59 AM,4616253,"40.8347204,-73.86593 40.83357,-73.86199 40.832...",_pfxF`}yaMdFsWfDmPpH}^lEgTBBBBB,BBBBB,NYC_DOT_LIC,4616253,Bronx,CBE E TAYLOR AVENUE - CASTLE HILL AVENUE
4,3,14.91,422,0,06/02/2017 11:41:59 PM,4616324,"40.76375,-73.999191 40.763521,-73.99935 40.762...",mtxwF|}sbMl@^~GpK|LrIbLlH??lK~G|FtD`C~@}@WdWnG...,BBBBBBBBBBBBBBB,NYC_DOT_LIC,4616324,Manhattan,12th ave @ 45th - 11 ave ganservoort st


In [10]:
cols = daskDF.columns.values
print(f'Given Columns: {cols}\n')
for i in range(len(daskDF.columns)):
    specialRemoved = ''.join([a for a in daskDF.columns[i] if (a.isalnum())])
    cols[i] = specialRemoved
dictCols = {i:cols[i] for i in range(len(cols))}
daskDF.rename(columns=dictCols)
print(f'Parsed Columns: {daskDF.columns.values.tolist()}')

Given Columns: ['ID' 'SPEED' 'TRAVEL_TIME' 'STATUS' 'DATA_AS_OF' 'LINK_ID' 'LINK_POINTS'
 'ENCODED_POLY_LINE' 'ENCODED_POLY_LINE_LVLS' 'OWNER' 'TRANSCOM_ID'
 'BOROUGH' 'LINK_NAME']

Parsed Columns: ['ID', 'SPEED', 'TRAVELTIME', 'STATUS', 'DATAASOF', 'LINKID', 'LINKPOINTS', 'ENCODEDPOLYLINE', 'ENCODEDPOLYLINELVLS', 'OWNER', 'TRANSCOMID', 'BOROUGH', 'LINKNAME']


YAML

In [11]:
import yaml

In [12]:
%%writefile data.yaml
fileType: csv
fileName: DOT_Traffic_Speeds_NBE
readDelimiter: ","
writeDelimiter: "|"
columns: ['ID', 'SPEED', 'TRAVELTIME', 'STATUS', 'DATAASOF', 'LINKID', 'LINKPOINTS', 'ENCODEDPOLYLINE', 'ENCODEDPOLYLINELVLS', 'OWNER', 'TRANSCOMID', 'BOROUGH', 'LINKNAME']

Overwriting data.yaml


In [13]:
data = open('data.yaml', 'r')
yml = yaml.load(data, yaml.Loader)
print(yml)
print('Number of Columns:', len(yml['columns']))

{'fileType': 'csv', 'fileName': 'DOT_Traffic_Speeds_NBE', 'readDelimiter': ',', 'writeDelimiter': '|', 'columns': ['ID', 'SPEED', 'TRAVELTIME', 'STATUS', 'DATAASOF', 'LINKID', 'LINKPOINTS', 'ENCODEDPOLYLINE', 'ENCODEDPOLYLINELVLS', 'OWNER', 'TRANSCOMID', 'BOROUGH', 'LINKNAME']}
Number of Columns: 13


File Writing & Summary

In [14]:
daskDF.to_csv(f"{yml['fileName']}.gz", compression='gzip', sep=yml['writeDelimiter'])

print(f"Number of Columns: {len(daskDF.columns.values)}")
print(f"Number of Rows: {len(daskDF)}")
print('File Size:', os.path.getsize(f"{yml['fileName']}.gz"))

Number of Columns: 13
Number of Rows: 64914523
File Size: 196608
